In [ ]:
from datetime import datetime, timedelta
import requests
import time
import hmac
import json
import hashlib
import pandas as pd
from shop_auth import shop_id,partner_id,host,key
import os

In [ ]:
def shop_auth(key,partner_id):
    timest = int(datetime.timestamp(datetime.now())) 
    path = "/api/v2/shop/auth_partner"
    redirect_url = "url_callbck"
    baseStr = str(partner_id) + path + str(timest)
    sign = hmac.new(key.encode('utf-8'), baseStr.encode('utf-8'), hashlib.sha256).hexdigest()
    #generate api
    url = host + path + "?partner_id=%s&timestamp=%s&sign=%s&redirect=%s" % (partner_id, timest, sign, redirect_url)
    print(url)
shop_auth(key,partner_id)

In [ ]:
auth = 'get_from_shop_auth'

In [ ]:
def get_token_shopee(code, shop_id,partner_id):
    ts = int(datetime.timestamp(datetime.now()))
    body = {"code": code, "shop_id": shop_id, "partner_id": partner_id}
    
    path = "/api/v2/auth/token/get"
    baseStr = str(partner_id) + path + str(ts)
    sign = hmac.new(key.encode('utf-8'), baseStr.encode('utf-8'), hashlib.sha256).hexdigest()

    url = host + path + "?partner_id=" + str(partner_id) + "&timestamp=" + str(ts) + "&sign=" + str(sign)
    headers = {"Content-Type" : "application/json"}
    response = requests.post(url, json=body, headers=headers)
    print(response.text)
    content = json.loads(response.content)
    access_token = content.get("access_token")
    refresh_token = content.get("refresh_token")
    print('-----------',access_token)
    print("**********",refresh_token)
    data = {
        "access_token": access_token,
        "refresh_token": refresh_token
    }

    # บันทึกข้อมูลลงในไฟล์ JSON
    with open('token_shopee.json', 'w') as json_file:
        json.dump(data, json_file)
        
get_token_shopee(auth, shop_id,partner_id)

In [ ]:
def re_token_shopee():
    with open('tokens.json', 'r') as response_file:
        data = json.load(response_file)

    # เข้าถึงข้อมูลใน JSON object ตามปกติ
    access_token = data['access_token']
    refresh_token = data['refresh_token']
    ts = int(datetime.timestamp(datetime.now()))
    body = {"shop_id": shop_id, "partner_id": partner_id, "refresh_token": refresh_token}
    
    path = "/api/v2/auth/access_token/get"
    baseStr = str(partner_id) + path + str(ts)
    sign = hmac.new(key.encode('utf-8'), baseStr.encode('utf-8'), hashlib.sha256).hexdigest()

    url = host + path + "?partner_id=" + str(partner_id) + "&timestamp=" + str(ts) + "&sign=" + str(sign)
    headers = {"Content-Type" : "application/json"}
    response = requests.post(url, json=body, headers=headers)
    content = json.loads(response.content)
    print(content)
    access_token = content.get("access_token")
    refresh_token = content.get("refresh_token")

    data = {
        "access_token": access_token,
        "refresh_token": refresh_token
    }

    with open("token_new.json", "w") as outfile:
        json.dump(data, outfile)

    with open('token_new.json', 'r') as response_file:
        data_new = json.load(response_file)
    access_token_new = data_new['access_token']
    refresh_token_new = data_new['refresh_token']

    return access_token_new,refresh_token_new

  
access_token_new,refresh_token_new = re_token_shopee()
print(access_token_new)

In [ ]:
def get_order_id(time_from,time_to):
    timest = int(datetime.timestamp(datetime.now())) 
    path = "/api/v2/order/get_order_list"
    baseStr = str(partner_id) + path + str(timest) + str(access_token_new) + str(shop_id)
    order_status = ['COMPLETED']
    # 'UNPAID','READY_TO_SHIP','PROCESSED','SHIPPED',
    sign = hmac.new(key.encode('utf-8'), baseStr.encode('utf-8'), hashlib.sha256).hexdigest()
    # partner_id, api path, timestamp, access_token, shop_id and partner_key
    df = pd.DataFrame()
    for i in order_status:
        # print(i)

        url = f"https://partner.shopeemobile.com{path}?access_token={access_token_new}&cursor=0&order_status={i}&page_size=100&partner_id={partner_id}&response_optional_fields=order_status&shop_id={shop_id}&sign={sign}&time_from={time_from}&time_range_field=update_time&time_to={time_to}&timestamp={timest}"


        # print(url)
        payload = {}
        headers = {}

        response = requests.request("GET",url,headers=headers, data=payload, allow_redirects=False)

        # print(response.text)
        print(response.text)
        y = json.loads(response.text)
        json_string = json.dumps(y, indent=4)

        # แปลงสตริง JSON เป็นโครงสร้างข้อมูล Python
        data = json.loads(json_string)

        df_raw = pd.json_normalize(data["response"]["order_list"])
        df = pd.concat([df,df_raw],ignore_index=True)
    print(len(df))
    return df

In [ ]:
def get_order_detail(order_id_list):
    timest = int(datetime.timestamp(datetime.now())) 
    path = "/api/v2/order/get_order_detail"
    baseStr = str(partner_id) + path + str(timest) + str(access_token_new) + str(shop_id)

    sign = hmac.new(key.encode('utf-8'), baseStr.encode('utf-8'), hashlib.sha256).hexdigest()
    order_detail = pd.DataFrame()
    item = pd.DataFrame()

    url = f"https://partner.shopeemobile.com{path}?access_token={access_token_new}&partner_id={partner_id}&shop_id={shop_id}&sign={sign}&timestamp={timest}&request_order_status_pending=true&response_optional_fields=order_sn,order_status,cancel_reason,response_optional_fields,buyer_username,create_time,pay_time,payment_method,shipping_carrier,ship_by_date,item_name,item_sku,model_original_price,model_discounted_price,model_quantity_purchased,total_amount,promotion_type,total_amount,estimated_shipping_fee,name,phone,message_to_seller,full_address,region,state,city,zipcode,order_status,update_time,note,recipient_address,order_list,item_list,checkout_shipping_carrier,package_list&order_sn_list={order_id_list}"

    payload=json.dumps(
        {
            "queries": [
            ]
        }
)
    headers = {
    'Content-Type': 'application/json'
    }
    
    response = requests.request("GET", url,headers=headers, data=payload, allow_redirects=False)

    print(response.text)
    json_data_order_detail = response.json()
    
    # แปลง Python dictionary กลับเป็น JSON โดยใช้ indent=2 เพื่อจัดรูปแบบ
    with open('get_order_detail.json', 'w') as outfile:
        json.dump(json_data_order_detail, outfile, indent=2)

    order_count = len(json_data_order_detail['response']['order_list'][0]['item_list'])


    order_detail_df = pd.DataFrame({
        'order_sn':[json_data_order_detail['response']['order_list'][0]['order_sn']],
        'buyer_username':[json_data_order_detail['response']['order_list'][0]['buyer_username']],
        'create_time':[json_data_order_detail['response']['order_list'][0]['create_time']],
        'order_status':[json_data_order_detail['response']['order_list'][0]['order_status']],
        'payment_method':[json_data_order_detail['response']['order_list'][0]['payment_method']],
        'pay_time':[json_data_order_detail['response']['order_list'][0]['pay_time']],
        'checkout_shipping_carrier':[json_data_order_detail['response']['order_list'][0]['checkout_shipping_carrier']],
        'shipping_carrier':[json_data_order_detail['response']['order_list'][0]['shipping_carrier']],
        'update_time':[json_data_order_detail['response']['order_list'][0]['update_time']],
    })

    order_detail = pd.concat([order_detail, order_detail_df], ignore_index=True)
    # print(order_count)
    for k in range(order_count):
        item_df = pd.DataFrame({
            'item_id':[json_data_order_detail['response']['order_list'][0]['item_list'][k]['item_id']],
            'item_name':[json_data_order_detail['response']['order_list'][0]['item_list'][k]['item_name']],
            'item_sku':[json_data_order_detail['response']['order_list'][0]['item_list'][k]['item_sku']],
            'model_id':[json_data_order_detail['response']['order_list'][0]['item_list'][k]['model_id']],
        })
        item = pd.concat([item, item_df], ignore_index=True)

    if order_count > 1:
        for i in range(order_count):
            new_row = order_detail.iloc[0].copy()
            order_detail = pd.concat([order_detail, pd.DataFrame([new_row])], ignore_index=True)
        order_detail.drop(order_detail.tail(1).index, inplace=True)
    


    df_merge = pd.merge(order_detail, item, left_index=True, right_index=True, how='inner')
    return df_merge
    


In [ ]:
def get_escrow_detail(order_id_list):
    timest = int(datetime.timestamp(datetime.now())) 
    path = "/api/v2/payment/get_escrow_detail"
    baseStr = str(partner_id) + path + str(timest) + str(access_token_new) + str(shop_id)
    get_escrow_detail = pd.DataFrame()
    item = pd.DataFrame()
    sign = hmac.new(key.encode('utf-8'), baseStr.encode('utf-8'), hashlib.sha256).hexdigest()

    url_get_escrow_detail = f"https://partner.shopeemobile.com{path}?access_token={access_token_new}&partner_id={partner_id}&shop_id={shop_id}&sign={sign}&timestamp={timest}&order_sn={order_id_list}"
    # print(order_id_list)
    payload__escrow=json.dumps({
    "queries": [
        {
        "order_sn": f"{order_id_list}"
        }
    ]
    })
    headers_escrow = {
    'Content-Type': 'application/json'
    }

    response_get_escrow_detail = requests.request("GET", url_get_escrow_detail, headers=headers_escrow, data=payload__escrow, allow_redirects=False)
    print(response_get_escrow_detail.text)
    json_data_get_escrow_detail = response_get_escrow_detail.json()

    # แปลง Python dictionary กลับเป็น JSON โดยใช้ indent=2 เพื่อจัดรูปแบบ
    with open('get_escrow_detail.json', 'w') as outfile:
        json.dump(json_data_get_escrow_detail, outfile, indent=2)
    count_item = len(json_data_get_escrow_detail['response']['order_income']['items'])
    main = pd.DataFrame({
        'order_sn': [json_data_get_escrow_detail['response']['order_sn']],
        'return_order_sn_list': [json_data_get_escrow_detail['response']['return_order_sn_list']],
        'buyer_user_name': [json_data_get_escrow_detail['response']['buyer_user_name']],
        'buyer_payment_method': [json_data_get_escrow_detail['response']['order_income']['buyer_payment_method']],
        'instalment_plan': [json_data_get_escrow_detail['response']['order_income']['instalment_plan']],
        'ค่าธรรมเนียม (%)' : '3.21%',
        'seller_return_refund':[json_data_get_escrow_detail['response']['order_income']['seller_return_refund']],
        'coins':[json_data_get_escrow_detail['response']['order_income']['coins']],
        'buyer_paid_shipping_fee':[json_data_get_escrow_detail['response']['order_income']['buyer_paid_shipping_fee']],
        'shopee_shipping_rebate':[json_data_get_escrow_detail['response']['order_income']['shopee_shipping_rebate']],
        'order_ams_commission_fee':[json_data_get_escrow_detail['response']['order_income']['order_ams_commission_fee']],
        'actual_shipping_fee':[json_data_get_escrow_detail['response']['order_income']['actual_shipping_fee']],
        'buyer_total_amount':[json_data_get_escrow_detail['response']['order_income']['buyer_total_amount']],
        'final_return_to_seller_shipping_fee':[json_data_get_escrow_detail['response']['order_income']['final_return_to_seller_shipping_fee']],
        'commission_fee':[json_data_get_escrow_detail['response']['order_income']['commission_fee']],
        'service_fee':[json_data_get_escrow_detail['response']['order_income']['service_fee']],
        'seller_transaction_fee':[json_data_get_escrow_detail['response']['order_income']['seller_transaction_fee']],
        'escrow_amount':[json_data_get_escrow_detail['response']['order_income']['escrow_amount']],
        'seller_voucher_code':[json_data_get_escrow_detail['response']['order_income']['seller_voucher_code']],
        'seller_lost_compensation':[json_data_get_escrow_detail['response']['order_income']['seller_lost_compensation']],
        'seller_shipping_discount':[json_data_get_escrow_detail['response']['order_income']['seller_shipping_discount']],
        'seller_coin_cash_back':[json_data_get_escrow_detail['response']['order_income']['seller_coin_cash_back']],
        'prorated_payment_channel_promo_shopee_offset_return_items':[json_data_get_escrow_detail['response']['order_income']['prorated_payment_channel_promo_shopee_offset_return_items']],
        'prorated_coins_value_offset_return_items':[json_data_get_escrow_detail['response']['order_income']['prorated_coins_value_offset_return_items']],
        'prorated_shopee_voucher_offset_return_items':[json_data_get_escrow_detail['response']['order_income']['prorated_shopee_voucher_offset_return_items']],
    })

        # ตรวจสอบว่า buyer_payment_method ไม่เป็นค่าว่าง และ instalment_plan ไม่เป็นค่าว่าง
    # ตรวจสอบว่า buyer_payment_method และ instalment_plan ไม่เป็นค่าว่าง
    if json_data_get_escrow_detail['response']['order_income']['buyer_payment_method'] == 'SPayLater' and \
        json_data_get_escrow_detail['response']['order_income']['instalment_plan'] is not None:
        # กำหนดค่าให้กับช่องทางการชำระเงิน (รายละเอียด) _1 เป็น 'Bell'
        main['ช่องทางการชำระเงิน (รายละเอียด) _1'] = json_data_get_escrow_detail['ช่องทางการชำระเงิน (รายละเอียด) _1'] = 'Shopee Credit Down Payment'

    

    get_escrow_detail = pd.concat([get_escrow_detail, main], ignore_index=True)
    for i in range(count_item):
        item_df = pd.DataFrame({
            'item_id':[json_data_get_escrow_detail['response']['order_income']['items'][i]['item_id']],
            'item_name':[json_data_get_escrow_detail['response']['order_income']['items'][i]['item_name']],
            'item_sku':[json_data_get_escrow_detail['response']['order_income']['items'][i]['item_sku']],
            'model_id':[json_data_get_escrow_detail['response']['order_income']['items'][i]['model_id']],
            'quantity_purchased':[json_data_get_escrow_detail['response']['order_income']['items'][i]['quantity_purchased']],
            'original_price':[json_data_get_escrow_detail['response']['order_income']['items'][i]['original_price']],
            'seller_discount':[json_data_get_escrow_detail['response']['order_income']['items'][i]['seller_discount']],
            'shopee_discount':[json_data_get_escrow_detail['response']['order_income']['items'][i]['shopee_discount']],
            'discount_from_voucher_seller':[json_data_get_escrow_detail['response']['order_income']['items'][i]['discount_from_voucher_seller']],
            'ams_commission_fee':[json_data_get_escrow_detail['response']['order_income']['items'][i]['ams_commission_fee']],
        })
        item = pd.concat([item, item_df], ignore_index=True)

    if count_item > 1:
        for i in range(count_item):
            new_row = get_escrow_detail.iloc[0].copy()
            get_escrow_detail = pd.concat([get_escrow_detail, pd.DataFrame([new_row])], ignore_index=True)
        get_escrow_detail.drop(get_escrow_detail.tail(1).index, inplace=True)
        pass

    df_merge = pd.merge(get_escrow_detail, item, left_index=True, right_index=True, how='inner')

    return df_merge


In [ ]:
def convert_unix_timestamp_to_date(timestamp):
    if timestamp is not None:
        date = datetime.utcfromtimestamp(timestamp + 25200)  # 7 ชั่วโมงในรูปแบบวินาที
        return date.strftime('%Y-%m-%d')
    else:
        return None

In [ ]:
def final_data_shopee(time_from,time_to):
    order_detail = pd.DataFrame()
    escrow_detail = pd.DataFrame()
    order_ids = get_order_id(time_from, time_to)
    order_id_list = order_ids['order_sn'].unique()

    # ประกาศตัวแปร order_sn ไว้ก่อนหน้าลูป
    for order_sn in order_id_list:
        # print(order_sn)
        row_order = get_order_detail(order_sn)
        order_detail = pd.concat([order_detail, row_order], ignore_index=True)

        row_escrow = get_escrow_detail(order_sn)
        escrow_detail = pd.concat([escrow_detail, row_escrow], ignore_index=True)

    merged_df = pd.merge(order_detail, escrow_detail, left_on=['order_sn','buyer_username','item_id','item_name','item_sku','model_id'], right_on=['order_sn','buyer_user_name','item_id','item_name','item_sku','model_id'], how='right')

    merged_df['create_time'] = merged_df['create_time'].apply(convert_unix_timestamp_to_date)

    merged_df['update_time'] = merged_df['update_time'].apply(convert_unix_timestamp_to_date)

    merged_df = merged_df[['order_sn','order_status','return_order_sn_list','buyer_username','create_time','payment_method','instalment_plan','ค่าธรรมเนียม (%)','update_time','item_id','item_name','item_sku','quantity_purchased','original_price','seller_discount','seller_return_refund','shopee_discount','discount_from_voucher_seller','seller_coin_cash_back','buyer_paid_shipping_fee','shopee_shipping_rebate','actual_shipping_fee','buyer_total_amount','final_return_to_seller_shipping_fee','order_ams_commission_fee','commission_fee','service_fee','seller_transaction_fee','escrow_amount','seller_voucher_code','seller_lost_compensation','seller_shipping_discount','checkout_shipping_carrier','shipping_carrier','prorated_coins_value_offset_return_items','prorated_payment_channel_promo_shopee_offset_return_items','prorated_shopee_voucher_offset_return_items']]


    merged_df['return_order_sn_list'] = merged_df['return_order_sn_list'].astype(str)
    merged_df['seller_voucher_code'] = merged_df['seller_voucher_code'].astype(str)
    # merged_df = merged_df.drop_duplicates()

    return merged_df


วันปัจจุบัน ย้อนไป 15 วัน

In [ ]:
current_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

# วันที่เมื่อวานตอนเที่ยงคืน
yesterday_date = current_date - timedelta(days=15)

# แปลงเป็น timestamp
time_to = int(datetime.timestamp(current_date))
time_from = int(datetime.timestamp(yesterday_date))

df = final_data_shopee(time_from,time_to)
df

รายวัน

In [ ]:
current_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

# วันที่เมื่อวานตอนเที่ยงคืน
yesterday_date = current_date - timedelta(days=1)
print(datetime.now())
print(current_date)
print(yesterday_date)

# # แปลงเป็น timestamp
time_to = int(datetime.timestamp(current_date))
time_from = int(datetime.timestamp(yesterday_date))


df = final_data_shopee(time_from,time_to)
df